In [0]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.optim as optim
from torchvision import transforms, models
import cv2



In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
def transform_image(img):
    
    img = torch.from_numpy(img.transpose((2, 0, 1))).float().to(device)
    mean = img.new_tensor([0.485, 0.456, 0.406]).view(-1, 1, 1).to(device)
	
    std = img.new_tensor([0.229, 0.224, 0.225]).view(-1, 1, 1).to(device)
    img = img.div_(255.0)
    img = (img - mean) / std
    return img.unsqueeze(0)

def transform_back_image(img):
    
    img = img.data.to(device)
    mean = img.new_tensor([0.485, 0.456, 0.406]).view(-1, 1, 1).to(device)
    std = img.new_tensor([0.229, 0.224, 0.225]).view(-1, 1, 1).to(device)
    img = img * std + mean
    img = img.clamp(0, 1)[0,:,:,:]
    img = img.cpu().numpy().transpose((1, 2, 0))
    img = img * 255.0
    return img.astype(np.uint8)

In [0]:
content_tensor = transform_image(cv2.resize(cv2.imread("/content/drive/My Drive/Colab Notebooks/dz_lost/images/Tuebingen_Neckarfront.jpg"), (500, 500)))
target = transform_image(cv2.resize(cv2.imread("/content/drive/My Drive/Colab Notebooks/dz_lost/images/Tuebingen_Neckarfront.jpg"), (500, 500))).requires_grad_(True)
styleA_tensor = transform_image(cv2.resize(cv2.imread("/content/drive/My Drive/Colab Notebooks/dz_lost/images/Vassily_Kandinsky_1913.jpg"), (500, 500)))
styleB_tensor = transform_image(cv2.resize( cv2.imread("/content/drive/My Drive/Colab Notebooks/dz_lost/images/The_Great_Wave_off_Kanagawa.jpg"), (500, 500)))


In [33]:
vgg = models.vgg19(pretrained=True).features
for param in vgg.parameters():
    param.requires_grad_(False)

vgg.to(device)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

In [0]:
def gram_matrix(feature): #матрица Грамма
    n, c, h, w = feature.size()
    feature = feature.view(c, h * w)
    gram = torch.mm(feature, feature.t())
    
    return gram 

In [0]:
content_features = Features(content_tensor, vgg)
styleA_features = Features(styleA_tensor, vgg)
styleA_grams = {layer: gram_matrix(styleA_features[layer]) for layer in styleA_features}

styleB_features = Features(styleB_tensor, vgg)
styleB_grams = {layer: gram_matrix(styleB_features[layer]) for layer in styleB_features}


In [0]:
def StyleLoss(target_features, style_grams, style_weights):
    style_loss = 0
    for layer in style_weights:
        target_feature = target_features[layer]
        target_gram = gram_matrix(target_feature)
        n, c, h, w = target_feature.shape

        style_gram = style_grams[layer]
        layer_style_loss = style_weights[layer] * torch.mean((target_gram - style_gram)**2)
        style_loss += layer_style_loss / (c * h * w)
    
    return style_loss 

In [0]:
def Features(image, model, layers = None):    
    if layers is None:
        layers = {'0': 'conv1_1',
                  '5': 'conv2_1', 
                  '10': 'conv3_1', 
                  '19': 'conv4_1',
                  '21': 'conv4_2',
                  '28': 'conv5_1'}
        
    features = {}
    x = image
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[layers[name]] = x
            
    return features

In [0]:
lamda = 0.4 #lamda указывает на долю стиля A
#веса
style_weights = {'conv1_1': 0.9,
                 'conv2_1': 0.9,
                 'conv3_1': 0.1, #0.2
                 'conv4_1': 0.1,#0.2
                 'conv5_1': 0.1}#0.2



show_every = 10
optimizer = optim.Adam([target], lr=0.01)
steps = 5000 #5000
content_weight = 2e6  # content_weight
style_weight = 2e6  # style_weight

In [40]:
for epoch in range(1, steps+1):
    target_features = Features(target, vgg)

    content_loss = torch.mean((target_features['conv4_2'] - content_features['conv4_2'])**2)

    styleA_loss = StyleLoss(target_features, styleA_grams, style_weights)
    styleB_loss = StyleLoss(target_features, styleB_grams, style_weights)
    style_loss = styleA_loss * lamda + styleB_loss * (1 - lamda)
        
    total_loss = content_weight * content_loss + style_weight * style_loss #считаем
    
    # update my target image
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    
    print(epoch, 'Total loss: ', total_loss.item())
    # display intermediate images and print the loss
    if  epoch % show_every == 0:
        #print('Total loss: ', total_loss.item())

        result = transform_back_image(target)
        cv2.imwrite("/content/drive/My Drive/Colab Notebooks/dz_lost/images/result-3.png", result)

result = transform_back_image(target)
cv2.imwrite("/content/drive/My Drive/Colab Notebooks/dz_lost/images/result-3.png", result)

Streaming output truncated to the last 5000 lines.
2 Total loss:  2846458880.0
3 Total loss:  2704546816.0
4 Total loss:  2566153984.0
5 Total loss:  2432836864.0
6 Total loss:  2306767360.0
7 Total loss:  2190318080.0
8 Total loss:  2085691392.0
9 Total loss:  1994478848.0
10 Total loss:  1917320320.0
11 Total loss:  1853607552.0
12 Total loss:  1801515136.0
13 Total loss:  1758310272.0
14 Total loss:  1721040384.0
15 Total loss:  1687176960.0
16 Total loss:  1654914432.0
17 Total loss:  1623199360.0
18 Total loss:  1591651840.0
19 Total loss:  1560336256.0
20 Total loss:  1529605120.0
21 Total loss:  1499857280.0
22 Total loss:  1471457024.0
23 Total loss:  1444653696.0
24 Total loss:  1419598208.0
25 Total loss:  1396329984.0
26 Total loss:  1374800384.0
27 Total loss:  1354884096.0
28 Total loss:  1336432384.0
29 Total loss:  1319295232.0
30 Total loss:  1303328384.0
31 Total loss:  1288403840.0
32 Total loss:  1274418048.0
33 Total loss:  1261296512.0
34 Total loss:  1248965120.0


True

In [43]:
plt.figure()
imshow(cv2, title='Output Image')
#plt.imsave(output, 'output.png')
# sphinx_gallery_thumbnail_number = 4
plt.ioff()
plt.show()

AttributeError: ignored

<Figure size 432x288 with 0 Axes>